In [ ]:
import anndata as ad
import scanpy as sc
#import squidpy as sq
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import cell2location as c2l
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore', category=UserWarning)
from dotenv import load_dotenv
load_dotenv()

In [ ]:
adata_scatlas = ad.read_h5ad(os.environ.get('PATH_TO_SCATLAS'))
adata_visium = ad.read_h5ad(os.environ.get('PATH_TO_VISIUM'))

# 1. Data Overview

In [ ]:
print("Number of cells before filtering:", adata_scatlas.n_obs)

Remove zero count genes (Korbinian approved)

In [ ]:
adata_scatlas.var_names = adata_scatlas.var["original_gene_names"]
sc.pp.filter_cells(adata_scatlas, min_counts=1)
sc.pl.highest_expr_genes(adata_scatlas, n_top=20)

In [ ]:
print("Number of cells after filtering out zero counts:", adata_scatlas.n_obs)

In [ ]:
adata_scatlas.obs["total_counts"] = adata_scatlas.X.sum(axis=1)
sns.histplot(adata_scatlas.obs['total_counts'])
plt.xlabel('Total Counts')
plt.ylabel('Cell Count')
plt.title('Distribution of Total Counts')

Ensure the original data was nomalized by reviewing the raw counts

In [ ]:
adata_scatlas.obs["total_counts_raw"] = adata_scatlas.raw.X.sum(axis=1)
sns.histplot(adata_scatlas.obs['total_counts_raw'])
plt.xlabel('Raw Total Counts')
plt.ylabel('Cell Count')
plt.title('Distribution of Raw Total Counts')

In [ ]:
sc.pl.umap(adata_scatlas, color = ["cell_type_level1","cell_type_level2"])

# 2. Train Model Plaque Atlas Data

In [ ]:
# Estimate some cut-off values for filtering
total_genes = adata_scatlas.shape[1]
print(f"Total number of genes: {total_genes}")

# Get the number of cells where each gene has non-zero expression
gene_cell_count = np.sum(adata_scatlas.X > 0, axis=0)

# Count the number of genes with more than 5 cell counts
genes_with_5_cells = np.sum(gene_cell_count > 5)
print(f"Number of genes with more than 5 cell counts: {genes_with_5_cells}")
print(f"Number of cells: {adata_scatlas.shape[0]}")

In [ ]:
from cell2location.utils.filtering import filter_genes
adata_ref = adata_scatlas.copy()
adata_ref.X = adata_ref.raw.X
selected = filter_genes(adata_ref, cell_count_cutoff=1, cell_percentage_cutoff2=0.03, nonz_mean_cutoff=1.12)
adata_ref = adata_ref[:, selected].copy()

# 3. Import Model

In [ ]:
model_path = os.environ.get('PATH_TO_MODEL')
model2wo = c2l.models.RegressionModel.load(model_path, adata_ref)
adata_ref = model2wo.export_posterior(
    adata_ref, use_quantiles=True,
    add_to_varm=["q05","q50", "q95"],
    sample_kwargs={'batch_size': 2500}
)
# For the QC plot afterwards
adata_ref = model2wo.export_posterior(adata_ref, sample_kwargs={'num_samples': 1000, 'batch_size': 2500})

In [ ]:
model2wo.plot_history(20)

In [ ]:
model2wo.plot_QC()

In [ ]:
# Extracting reference cell types signatures as a pd.DataFrame
if 'means_per_cluster_mu_fg' in adata_ref.varm.keys():
    inf_aver = adata_ref.varm['means_per_cluster_mu_fg'][[f'means_per_cluster_mu_fg_{i}'
                                    for i in adata_ref.uns['mod']['factor_names']]].copy()
else:
    inf_aver = adata_ref.var[[f'means_per_cluster_mu_fg_{i}'
                                    for i in adata_ref.uns['mod']['factor_names']]].copy()
inf_aver.columns = adata_ref.uns['mod']['factor_names']
inf_aver

# 4. Cell type mapping

In [ ]:
adata_visium.obs['sample']

In [ ]:
def select_slide(adata, s, s_col='sample'):
    r""" This function selects the data for one slide from the spatial anndata object.

    :param adata: Anndata object with multiple spatial experiments
    :param s: name of selected experiment
    :param s_col: column in adata.obs listing experiment name for each location
    """

    slide = adata[adata.obs[s_col].isin([s]), :]
    s_keys = list(slide.uns['spatial'].keys())
    s_spatial = np.array(s_keys)[[s in k for k in s_keys]][0]

    slide.uns['spatial'] = {s_spatial: slide.uns['spatial'][s_spatial]}

    return slide

In [ ]:
Sample_to_Disease = dict(
    FW104302="Fibroatheroma",
    FW104306="Fibroatheroma",
    FW104860="Atheroma",
    FW106005_v2="Atheroma",
    FW106006="Intermediate lesion",
    FW106008="Intermediate lesion",
    FW106010="Control",
    FW106012="Control",
    FW106014="Atheroma",
    FW106016="Atheroma",
    FW106018="Control",
    FW106022="Atheroma")
adata_visium.obs['Disease'] = (
    adata_visium.obs['sample'].map(Sample_to_Disease)
)
# Sort samples by disease group
sorted_samples = adata_visium.obs.sort_values(by='Disease')['sample'].unique()

# PLOT QC FOR EACH SAMPLE

fig, axs = plt.subplots(len(sorted_samples), 2, figsize=(7, len(sorted_samples)*3))

# Define color palette for disease categories
disease_palette = {'Control': '#52006A', 'Intermediate lesion': '#FF7600', 
                   'Atheroma': '#CD113B', 'Fibroatheroma': '#808080'}

for i, s in enumerate(sorted_samples):
    slide = select_slide(adata_visium, s)
    sample_value = adata_visium.obs.loc[adata_visium.obs['sample'] == s, 'sample'].unique()[0]
    disease_value = adata_visium.obs.loc[adata_visium.obs['sample'] == s, 'Disease'].unique()[0]
    header = f'{sample_value} | {disease_value}'

    # Get color for the disease category
    disease_color = disease_palette.get(disease_value, 'gray')

    # Plot the distribution of total_counts
    sns.histplot(slide.obs['total_counts'], kde=False, ax=axs[i, 0], color=disease_color, bins=30)
    axs[i, 0].set_xlim(0, adata_visium.obs['total_counts'].max())
    axs[i, 0].set_xlabel('total_counts')
    axs[i, 0].set_title(header)

    # Plot the distribution of n_genes_by_counts
    sns.histplot(slide.obs['n_genes_by_counts'], kde=False, ax=axs[i, 1], color=disease_color, bins=60)
    axs[i, 1].set_xlim(0, adata_visium.obs['n_genes_by_counts'].max())
    axs[i, 1].set_xlabel('n_genes_by_counts')
    axs[i, 1].set_title(header)

plt.tight_layout()
plt.show()

In [ ]:
sc.pl.umap(adata_visium, color=['sample'], size=30, color_map = 'RdPu', ncols = 1, legend_fontsize=10)

In [ ]:
# Plot UMAP with log-transformed data

# Log-transform (log(data + 1))
adata_vis_plt = adata_visium.copy()
sc.pp.log1p(adata_vis_plt)

# Find highly variable genes within each sample
adata_vis_plt.var['highly_variable'] = False
for s in adata_vis_plt.obs['sample'].unique():
    adata_vis_plt_1 = adata_vis_plt[adata_vis_plt.obs['sample'] == s, :]
    sc.pp.highly_variable_genes(adata_vis_plt_1, n_top_genes=3000)  
    hvg_list = list(adata_vis_plt_1.var_names[adata_vis_plt_1.var['highly_variable']])
    adata_vis_plt.var.loc[hvg_list, 'highly_variable'] = True

# Scale the data
sc.pp.scale(adata_vis_plt, max_value=10)

# PCA, KNN construction, UMAP
sc.tl.pca(adata_vis_plt, svd_solver='arpack', n_comps=40, mask_var='highly_variable')  
sc.pp.neighbors(adata_vis_plt, n_neighbors=25, n_pcs=40, metric='cosine')
sc.tl.umap(adata_vis_plt, min_dist=0.3, spread=1)

# Plot UMAP
sc.pl.umap(adata_vis_plt, color=['sample'], size=30, color_map='RdPu', ncols=1, legend_fontsize=10)

In [ ]:
adata_visium.var

In [ ]:
# find shared genes and subset both anndata and reference signatures
intersect = np.intersect1d(adata_visium.var_names, inf_aver.index)
adata_visium = adata_visium[:, intersect].copy()
inf_aver = inf_aver.loc[intersect, :].copy()

inf_aver

In [ ]:
# Since we are interested in Pro-Angiogenic EC
inf_aver.sort_values('Pro-Angiogenic EC', ascending=False).head(20)

In [ ]:
# Plot correlation of different cell types, using spearman rank method
corr = inf_aver.corr(method="spearman")

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

f, ax = plt.subplots(figsize=(11, 9))
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(corr, mask=mask, cmap=cmap, center=0.5,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [ ]:
# prepare anndata for cell2location model
c2l.models.Cell2location.setup_anndata(adata=adata_visium, batch_key="sample")

map_model2wo = c2l.models.Cell2location(
    adata_visium, cell_state_df=inf_aver,
    # the expected average cell abundance: tissue-dependent hyper-prior which can be estimated from paired histology:
    N_cells_per_location=5,
    # hyperparameter controlling normalisation of within-experiment variation in RNA detection:
    detection_alpha=200
)
map_model2wo.view_anndata_setup()